In [1]:
from tensorflow.keras import datasets
import numpy as np
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data()

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\jared\Anaconda3\envs\blue-mars\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\jared\Anaconda3\envs\blue-mars\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or sha

In [3]:
print(type(x_train), x_train.shape, y_train.shape)
print(type(x_test), x_test.shape, y_test.shape)

<class 'numpy.ndarray'> (25000,) (25000,)
<class 'numpy.ndarray'> (25000,) (25000,)


In [4]:
stoi = datasets.imdb.get_word_index()

In [5]:
print(type(stoi))
print(list(stoi.keys())[0])
print(list(stoi.values())[0])

<class 'dict'>
fawn
34701


In [57]:
print(x_train[0][:10])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
